In [ ]:
### **Code to Generate Hedging Accuracy Table**
import pandas as pd
import numpy as np
from google.colab import files
import io
import os
from sklearn.metrics import accuracy_score

# ===================================================================
# PART 1: RELOAD DATA TO GET THE THRESHOLD
# ===================================================================
# We need to reload the original data to find the 75th percentile
# of the *original* training set.

file_name = 'final_predictors_2003_onwards.csv'
if not os.path.exists(file_name):
    print(f"Please upload '{file_name}' to calculate the threshold.")
    uploaded = files.upload()
    filename_full = next(iter(uploaded))
else:
    print(f"Using cached file: '{file_name}'")
    filename_full = file_name

df = pd.read_csv(filename_full, index_col='Date', parse_dates=True)

# Re-create the target variable and do the split
df['USDINR_log_return'] = np.log(df['USDINR'] / df['USDINR'].shift(1))
df['REALIZED_VOLATILITY'] = df['USDINR_log_return'].rolling(window=21).std() * np.sqrt(252)
df.dropna(inplace=True)
y = df['REALIZED_VOLATILITY']
split_index = int(len(df) * 0.8)
train_df = df.iloc[:split_index]

# Get the threshold
volatility_threshold = y.loc[train_df.index].quantile(0.75)
print(f"Hedging Threshold (75th percentile of train vol): {volatility_threshold:.6f}")


# ===================================================================
# PART 2: LOAD RESULTS AND RUN HEDGING SIMULATION
# ===================================================================
results_file = 'results_Baseline.csv'
if not os.path.exists(results_file):
    print(f"Error: '{results_file}' not found.")
    print("Please run the main ablation script (Run 1: Baseline) first.")
else:
    print(f"Loading results from '{results_file}'...")
    results_df = pd.read_csv(results_file, index_col='Date', parse_dates=True)

    # --- 1. Create the "Actual" Decision ---
    results_df['Actual_Hedge'] = results_df['REALIZED_VOLATILITY'] > volatility_threshold

    # --- 2. Create Each Model's Decision ---
    # GARCH models were not in this file, so we only evaluate the models that are
    models_to_evaluate = ["HAR_Forecast", "LSTM_Full_Forecast", "LSTM_Simple_Forecast"]
    for model_name in models_to_evaluate:
        results_df[f'{model_name}_Hedge'] = results_df[model_name] > volatility_threshold

    # --- 3. Calculate and Print the Accuracy Table ---
    decision_metrics = {'Model': [], 'Hedge_Accuracy': []}
    for model_name in models_to_evaluate:
        # Use accuracy_score to see how often the model's decision matched the actual decision.
        accuracy = accuracy_score(results_df['Actual_Hedge'], results_df[f'{model_name}_Hedge'])
        decision_metrics['Model'].append(model_name)
        decision_metrics['Hedge_Accuracy'].append(accuracy * 100) # As percentage

    decision_metrics_df = pd.DataFrame(decision_metrics)
    print("\n--- Hedging Decision Accuracy (Baseline Run) ---")
    print(decision_metrics_df.round(2))